# How to Detect Light Bars

In [5]:
import cv2
import numpy as np
import imutils
print(cv2.__version__)

3.4.0


## Declare the Source

In [2]:
cap = cv2.VideoCapture('RedCar.avi') # Set to integer starting at 0 to use peripheral cameras

if not cap.isOpened():
    print('Error opening video stream or file')

Error opening video stream or file


## Play the Video

In [3]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        
        cv2.imshow('Frame', frame)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break

cap.release()
cv2.destroyAllWindows()

Error opening video stream or file


## Convert To HSV

### What's HSV?

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/HSV_color_solid_cylinder_alpha_lowgamma.png/800px-HSV_color_solid_cylinder_alpha_lowgamma.png)

In [4]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        cv2.imshow('HSV', hsv)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

Error opening video stream or file


## Detect Lights From HSV

In [237]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        cv2.imshow('Lights', lights)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Convert Back to BGR

Convert back to BGR for further manipulation

In [238]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        
        cv2.imshow('BGR Lights', lights)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Blur

In [239]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)    
        
        cv2.imshow('Blurred', blurred)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Threshold

In [240]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)
        
        thresh = cv2.threshold(blurred, 50, 255, cv2.THRESH_BINARY)[1]
        
        
        
        cv2.imshow('Threshold', thresh)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Define Detect Shapes Function

In [6]:
def detect(c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
 
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
 
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            if ar >= 0.95 and ar <= 1.05:
                shape = "square"
            elif ar <= 0.50:
                shape = "lightbar"
            else:
                shape = "rectangle"
                 
        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"
 
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
 
        # return the name of the shape
        return (shape, approx)

## Draw Shapes on Threshold

In [242]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)
        
        thresh = cv2.threshold(blurred, 30, 255, cv2.THRESH_BINARY)[1]
        
        gray = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
        
        
        cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        
        cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        
        
        for c in cnts:
            try:
                # compute the center of the contour, then detect the name of the
                # shape using only the contour
                M = cv2.moments(c)
                cX = int((M["m10"] / M["m00"]))
                cY = int((M["m01"] / M["m00"]))
                
                shape, approx = detect(c)


                if shape == 'lightbar':
                    cv2.drawContours(thresh, [c], -1, (0, 255, 0), 2)
                    cv2.putText(thresh, 'lightbar', (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    #draw bisecting lines
                    (x, y, w, h) = cv2.boundingRect(approx)
                    cv2.line(thresh, (int((x+(w/2))), y), (int((x+(w/2))), y+h), (255,0,0), 2)


            except:
                pass
        
        
        
        cv2.imshow('Threshold', thresh)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()

## Draw Shapes on Original Frame

In [7]:
cap = cv2.VideoCapture('RedCar.avi')

if not cap.isOpened():
    print('Error opening video stream or file')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        lower = np.array([0,0,251])
        upper = np.array([180,3,255])
        mask = cv2.inRange(hsv, lower, upper)
        
        lights = cv2.bitwise_and(hsv, hsv, mask=mask)
        
        lights_bgr = cv2.cvtColor(lights, cv2.COLOR_HSV2BGR)
        blurred = cv2.GaussianBlur(lights_bgr, (5,5), 0)
        
        thresh = cv2.threshold(blurred, 30, 255, cv2.THRESH_BINARY)[1]
        
        gray = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
        
        
        cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        
        cnts = cnts[0] if imutils.is_cv2() else cnts[1]
        
        prev_coords = None
        
        
        for c in cnts:
            try:
                # compute the center of the contour, then detect the name of the
                # shape using only the contour
                M = cv2.moments(c)
                cX = int((M["m10"] / M["m00"]))
                cY = int((M["m01"] / M["m00"]))
                
                shape, approx = detect(c)


                if shape == 'lightbar':
                    cv2.drawContours(frame, [c], -1, (0, 255, 0), 2)
                    cv2.putText(frame, 'lightbar', (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    #draw bisecting lines
                    (x, y, w, h) = cv2.boundingRect(approx)
                    cv2.line(frame, (int((x+(w/2))), y), (int((x+(w/2))), y+h), (255,0,0), 2)


            except:
                pass
        
        
        
        cv2.imshow('Output', frame)
        
    if (cv2.waitKey(25) & 0xFF == ord('q')) or not ret:
        break
        
cap.release()
cv2.destroyAllWindows()